In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd
import string
import csv

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
chrome_options.add_argument('--headless')
service = Service("/Users/tumendemberelshalkhaan/Downloads/chromedriver")
driver = webdriver.Chrome(service=service, options = chrome_options)
driver.get('http://www.doc.state.al.us/inmatesearch')

In [ ]:
ids = []

for letter in string.ascii_lowercase:

    driver.find_element('name', 'ctl00$MainContent$txtLName').send_keys(letter)
    driver.find_element('name', 'ctl00$MainContent$btnSearch').click()
    current = int(driver.find_element('id', 'MainContent_gvInmateResults_lblCurrent').text)
    pages = int(driver.find_element('id', 'MainContent_gvInmateResults_lblPages').text)
    ratio = current/pages

    while ratio < 1:
        next = driver.find_element("id", 'MainContent_gvInmateResults_btnNext')
        table = driver.find_element("id", "MainContent_gvInmateResults")
        rows = table.find_elements("xpath", ".//tr")
        for row in rows[:-1]:
            cells = row.find_elements("tag name", "td")
            if cells:
                ids.append(cells[0].text)
        current = int(driver.find_element('id', 'MainContent_gvInmateResults_lblCurrent').text)
        pages = int(driver.find_element('id', 'MainContent_gvInmateResults_lblPages').text)
        ratio = current/pages
        next.click()
    driver.get('http://www.doc.state.al.us/inmatesearch')

In [ ]:
al = pd.DataFrame(columns=['ID', 'Name', 'Race', 'Date Of Birth', 'Facility', 'Gender', 'Offense Description', 'Sentence Length', 'County'])

In [ ]:
with open('/Users/tumendemberelshalkhaan/Desktop/Alabama_DOC.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for element in ids:
        writer.writerow([element])

In [ ]:
import requests
from lxml import html

In [ ]:
failed_ids2 = []

for id in failed_ids2:
    try:
        driver.find_element('name', 'ctl00$MainContent$txtAIS').send_keys(id) 
        driver.find_element('name', 'ctl00$MainContent$btnSearch').click()
        driver.find_element('id', 'MainContent_gvInmateResults_lnkInmateName_0').click()
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
        name = soup.find('td', text = 'Inmate: ').find_next_sibling().get_text()
        fac = soup.find('td', text = 'Institution: ').find_next_sibling().get_text()
        race = soup.find('td', text = 'Race:').find_next_sibling().get_text()
        gender = soup.find('td', text = 'Sex:').find_next_sibling().get_text()
        dob = soup.find('td', text = 'Birth Year:').find_next_sibling().get_text()

        offenses = []
        sentences = []
        sendates = []
        counties = []
        table = soup.find('table', id = 'MainContent_gvSentence_GridView1_0').find()
        
        for row in table.find_all('tr')[1:]:
            data = [cell.get_text(strip=True) for cell in row.find_all('td')]
            offenses.append(data[2])
            sentences.append(data[3])
            counties.append(data[5])

        histories = [list(history) for history in zip(offenses, sentences, counties)]

        rows = []

        for history in histories:
            row = {
                'ID': id,
                'Name': name,
                'Date Of Birth': dob,
                'Gender': gender,
                'Race': race,
                'Facility': fac,
                'Offense Description': history[0],
                'Sentence Length': history[1],
                'County': history[2]
                }
            rows.append(row)

        al = pd.concat([al, pd.DataFrame(rows)])
        driver.get('http://www.doc.state.al.us/inmatesearch')
        time.sleep(2)   
    except Exception as e:
        print(f"Failed to scrape {id}. Error: {e}")
        failed_ids2.append(id)
        continue


In [ ]:
"javascript:__doPostBack('ctl00$MainContent$gvInmateResults$ctl04$lnkInmateName','')"

In [ ]:
al.to_csv('/Users/tumendemberelshalkhaan/Desktop/Alabama_Raw.csv', index=False)